In [ ]:
# Third-party imports
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u

# Project import
import artpop

# artpop's matplotlib style
plt.style.use(artpop.mpl_style)

# use this random state for reproducibility
rng = np.random.RandomState(122)

plt.rcParams['figure.figsize'] = (12, 9) #set figure size


<h3> Creating a Stellar Population </h3>

In [ ]:
# Example of initializing a stellar population
# Note that you want enough stars to adequately sample the IMF 
# but not so much that you use a ton of memory!
# Increasing num_stars will use more memory so be careful if you try to change this!

ssp = artpop.MISTSSP(
    log_age = 9,          # log of age in years 
    feh = 0,             # metallicity [Fe/H]
    phot_system = 'LSST', # photometric system(s)
    num_stars = 1e5,      # number of stars.
    imf = 'salpeter',       # default imf
    random_state = rng,   # random state for reproducibility
)

In [ ]:
# We can make an HR diagram (Luminosity vs Temperature) for our stars

#These select different evolutionary phases

MS = ssp.select_phase('MS') #Main Sequence
RGB = ssp.select_phase('RGB') #Red Giant Branch
AGB = ssp.select_phase('AGB') #Asymptotic Giant Branch

#Plot L vs Temp, coloring each phase separately

plt.plot(ssp.log_Teff[MS], ssp.log_L[MS], marker='o',ls='',
         label='MS', c='tab:blue', mec='k')
plt.plot(ssp.log_Teff[RGB], ssp.log_L[RGB], marker='o',ls='',
         label='RGB + subgiant branch',
         c='tab:green', mec='k')
plt.plot(ssp.log_Teff[AGB], ssp.log_L[AGB], marker='o',ls='',
         label='AGB', c='tab:red', mec='k')

plt.legend(loc='lower left')
plt.gca().invert_xaxis()
plt.minorticks_on()
plt.xlabel(r'$\log(\mathrm{T_{eff}/K})$')
plt.ylabel(r'$\log(\mathrm{L/L_\odot})$');

<h3> Calculating the Mass-to-Light Ratio </h3>

In [ ]:
# Mass-to-Light Ratio
# Total Mass M / Total Luminosity L
# When done in terms of the total luminosity (rather than just one band) this is called
# The BOLOMETRIC mass-to-light ratio

mass_to_light = np.sum(ssp.star_masses) / np.sum(10**ssp.log_L)

print(mass_to_light)

In [27]:
# The mass to light ratio for different photometric bands
# This dataset includes different bands that will be used in a future survey
# Called the Legacy Survey of Space and Time to be performed with the Vera Rubin Observatory
# Within the next 5 years or so

# You can see the available bands like this (they are u, g, r, i, z, and y from bluest to reddest)
# Note that "abs" stands for absolute so these are the true stellar magnitudes
ssp.abs_mags.keys()

# This is a dictionary, with each value representing a list of values corresponding to each of your stars
# To access the u-band magnitudes you would simply do the following, inputting the appropriate string
# in square brackets ([])

print('list of u-band magnitudes:', ssp.abs_mags['LSST_u'])

#Note that the length of the array should equal the number of stars

print('number of u-band magnitudes we have (sane as num_stars!):', len(ssp.abs_mags['LSST_u']))


# To calculate a mass-to-light ratio we need to sum up the contributions from each star
# ArtPop can do this for us, but in terms of magnitudes

mag_u_tot = ssp.total_mag('LSST_u')

#Now we want to take this relative to the sun
# recall that magnitudes m = -2.5log(f/frel)
# So if we know msun m - msun = -2.5[log(f/fsun)] = -2.5log(L/Lsun)

# In the LSST bands here are the values for the sun

mag_sun = {'LSST_u': 6.27, 'LSST_g': 5.06, 'LSST_r': 4.64, 
           'LSST_i': 4.52, 'LSST_z': 4.51, 'LSST_y': 4.50}

#So, we can apply this correction to the total magnitude

mag_u_tot_sun = mag_u_tot - mag_sun['LSST_u']

# Finally, we can convert this to a luminosity (in units of solar luminosity)

Lum_u = 10**(mag_u_tot_sun/-2.5)

# And now we can compare with the total mass for the ratio M/L

print("M/L in the LSST_u band:", np.sum(ssp.star_masses)/Lum_u)

#Now we can try this in a different band, repeating the steps above but for the LSST_i band

mag_i_tot = ssp.total_mag('LSST_i')
mag_i_tot_sun = mag_i_tot - mag_sun['LSST_i']
Lum_i = 10**(mag_i_tot_sun/-2.5)

print("M/L in the LSST_i band:", np.sum(ssp.star_masses)/Lum_i)




list of u-band magnitudes: [18.58289929 15.36870569 18.03560959 ... 15.81330171 17.20464547
 17.10463934]
number of u-band magnitudes we have (sane as num_stars!): 100000
M/L in the LSST_u band: 0.7098129159485961
M/L in the LSST_i band: 0.7347635619354883


<h3> Looking at how Mass-to-light ratio changes with age </h3>

In [ ]:
# We can calculate stellar populations for different stellar ages by creating a "GRID"
# of stellar populations, each sampling the same IMF but with different ages
# each element of ssp_grid is a separate population of a different age
# i.e. ssp_grid[2] will be the population with log(age) = 8

ssp_grid = []
log_age = [6,7,8,9,9.5,10]
for i in range(len(log_age)):
    ssp_grid.append(artpop.MISTSSP( #keep adding onto our grid as we loop through our chosen ages
                    log_age = log_age[i],   # log of age in years, looping through the list above
                    feh = 0,             # metallicity [Fe/H]
                    phot_system = 'LSST', # photometric system(s)
                    num_stars = 1e5,      # number of stars.
                    imf = 'salpeter',       # default imf
                    random_state = rng,   # random state for reproducibility
                    ))


In [ ]:
# Now we can calculate the total (or bolometric) mass to light ratio vs age

mass_tot = np.zeros(len(ssp_grid)) #create arrays to store our data
lum_tot = np.zeros(len(ssp_grid))
for i in range(len(ssp_grid)): #Loop over our grid
    mass_tot[i] = np.sum(ssp_grid[i].star_masses) #sum of all stellar masses 
    lum_tot[i] = np.sum(10**ssp_grid[i].log_L) #sum of all luminosities

mtol = mass_tot/lum_tot #final step! M/L

plt.plot(log_age, mtol, marker='o', ls='-') #now plot
plt.yscale('log')
plt.ylabel('Bolometric M/L')
plt.xlabel('log Age [yr]')


In [ ]:
# We can understand a bit why this evolution occurs
# By Plotting total mass and luminosity vs time
# Note that the change in luminosity is driving this evolution!
# It decreases by a factor of more than 1000 over 10 Gyr while
# mass decreases by about 30%

plt.plot(log_age, mass_tot/mass_tot[0], marker='o', ls='-', color='b')
#plt.yscale('log')
plt.ylim(0,1)
plt.ylabel('M(t)/M(t=1e6)', color='b')
plt.xlabel('log Age [yr]')
plt.twinx()
plt.plot(log_age, lum_tot/lum_tot[0], marker='^', ls='-',color='r')
plt.yscale('log')
plt.ylim(1e-4,1)
plt.ylabel('L(t)/L(t=1e6)',color='r')
plt.tick_params(axis='y')

In [ ]:
# Now make a plot of the mass to light ratio in three different bands:
# LSST_u, LSST_r, and LSST_i
# Follow the same steps above. 
# In this case, you want to create three new luminosity arrays (the mass is the same)
# Now your plot should have three lines with different colors. 
# Use the 'label' keyword in plot() to label with each band
# (e.g. label='u-band') and don't forget to put in a legend with plt.legend().

# Is there a difference? Which bands are less sensitive to stellar ages?

#Don't forget to include these commands:
#plt.legend() # create legend
#plt.yscale('log') # scale y axis
#plt.ylabel('M/L') # label both x and y axes
#plt.xlabel('log Age [yr]')

<h3> Mass-to-light Ratio and IMF </h3>

In [ ]:
# Now we can see how the stellar populations change with Initial Mass Function
# We wills tart by looking at populations that are 1 Gyr old and
# making a GRID of IMF slopes

alpha = [1.5,2.35,2.75,3.5,4.5] # here are our different slopes (dN/dM = M^-alpha)
imf_ssp_grids = []
for j in range(len(alpha)):
    print('making SSP for alpha =', alpha[j], 'and age 1 Gyr')
    imf_ssp_grids.append(artpop.MISTSSP(
                log_age = 9,   # log of age in years, looping through the list above
                feh = 0,             # metallicity [Fe/H]
                phot_system = 'LSST', # photometric system(s)
                num_stars = 1e5,      # number of stars.
                imf = {'a':-1*alpha[j]},       # new imf slope!
                random_state = rng,   # random state for reproducibility
                ))

In [ ]:
# Now we can calculate the total (or bolometric) mass to light ratio vs metalicity

mass_tot = np.zeros(len(imf_ssp_grids)) #create arrays to store our data
lum_tot = np.zeros(len(imf_ssp_grids))
for i in range(len(imf_ssp_grids)): #Loop over our grid
    mass_tot[i] = np.sum(imf_ssp_grids[i].star_masses) #sum of all stellar masses 
    lum_tot[i] = np.sum(10**imf_ssp_grids[i].log_L) #sum of all luminosities

mtol = mass_tot/lum_tot #final step! M/L

plt.plot(alpha, mtol, marker='o', ls='-') #now plot
plt.yscale('log')
plt.ylabel('Bolometric M/L')
plt.xlabel('IMF Slope')

In [ ]:
# Create three new grids of stellar populations but for log age = 6, 8, and 10
# Just copy and paste code used to make the different alpha models and re-do it for 
# The different ages (don't forget to give the variables different names!)

#For example, here is one for log Age = 10 (10^10 years or 10 Gyrs)

alpha = [1.5,2.35,2.75,3.5,4.5] # here are our different slopes (dN/dM = M^-alpha)
imf_ssp_grids_10 = []
for j in range(len(alpha)):
    print('making SSP for alpha =', alpha[j], 'and age 10 Gyr')
    imf_ssp_grids_10.append(artpop.MISTSSP(
                log_age = 10,   # log of age in years, looping through the list above
                feh = 0,             # metallicity [Fe/H]
                phot_system = 'LSST', # photometric system(s)
                num_stars = 1e5,      # number of stars.
                imf = {'a':-1*alpha[j]},       # new imf slope!
                random_state = rng,   # random state for reproducibility
                ))
    

In [ ]:
# Then plot the results (M/L vs Alpha) for all four ages (log age = 6, 8, 9, and 10)
# Your plot should have four lines, each a different color and each LABELED appropriately


<h3> Mass-to-light Ratio and Metalicity </h3>

In [ ]:
# Similar to above, we can also examine how M/L changes with
# the composition of our stars
# The example below creates such a grid at age 10^9 years and alpha=2.35

feh = [-2,-1,-0.5, 0.0, 0.5] # here are our different metalicities
Z_ssp_grids = []
for j in range(len(alpha)):
    print('making SSP for Z =', feh[j], 'and age 1 Gyr')
    Z_ssp_grids.append(artpop.MISTSSP(
                log_age = 9,   # log of age in years, looping through the list above
                feh = feh[j],             # metallicity [Fe/H]
                phot_system = 'LSST', # photometric system(s)
                num_stars = 1e5,      # number of stars.
                imf = 'salpeter',       # new imf slope!
                random_state = rng,   # random state for reproducibility
                ))

In [ ]:
# Just as above, we can plot these values

mass_tot = np.zeros(len(Z_ssp_grids)) #create arrays to store our data
lum_tot = np.zeros(len(Z_ssp_grids))
for i in range(len(Z_ssp_grids)): #Loop over our grid
    mass_tot[i] = np.sum(Z_ssp_grids[i].star_masses) #sum of all stellar masses 
    lum_tot[i] = np.sum(10**Z_ssp_grids[i].log_L) #sum of all luminosities

mtol = mass_tot/lum_tot #final step! M/L

plt.plot(feh, mtol, marker='o', ls='-') #now plot
plt.ylim(0,1)
#plt.yscale('log')
plt.ylabel('Bolometric M/L')
plt.xlabel('Metalicity [Fe/h]')

In [ ]:
# Now create three new grids for log ages = 6, 8, and 10

# Once again, to start you off, here is a grid for log age = 10
feh = [-2,-1,-0.5, 0.0, 0.5] # here are our different metalicities
Z_ssp_grids_10 = []
for j in range(len(alpha)):
    print('making SSP for feh =', feh[j], 'and age 10 Gyr')
    Z_ssp_grids_10.append(artpop.MISTSSP(
                log_age = 10,   # log of age in years, looping through the list above
                feh = feh[j],             # metallicity [Fe/H]
                phot_system = 'LSST', # photometric system(s)
                num_stars = 1e5,      # number of stars.
                imf = 'salpeter',       # new imf slope!
                random_state = rng,   # random state for reproducibility
                ))

In [ ]:
# Then plot the results (M/L vs feh) for all four ages (log age = 6, 8, 9, and 10)
# Your plot should have four lines, each a different color and each LABELED appropriately


In [ ]:
# You now have grids of IMF and metalicity for various population ages
# Using these, examine the mass-to-light ratio in different bands
# (remember the available bands are (in wavelength order low->high):
# 'LSST_u', 'LSST_g', 'LSST_r', 'LSST_i', 'LSST_z', 'LSST_y'

# Make plots to answer the following:
# Which bandpasses are most/least sensitive to stellar age, IMF, and metalicity?

# Note: Don't Forget to correctly label your axes and your lines!


In [ ]:
# Consider now the colors produced by different stellar populations
# Rather than luminosity you now want to examine the total magnitude of your stars
# You can use the total_mag() function (see example above) to do this

# Make plots to describe how colors change with age, IMF, and metalicity
# Do this for U - R and G - I to start (you can add more if you wish)